In [1]:
import transformers

In [2]:
import tensorflow as tf

In [3]:
from transformers import pipeline

In [4]:
classifier = pipeline("sentiment-analysis")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [6]:
classifier("Je suis vraiment content de suivre ce cours")

[{'label': 'NEGATIVE', 'score': 0.7081433534622192}]